In [13]:
from transformers import BertTokenizer, TFBertModel,BertForQuestionAnswering,BertForMaskedLM,BertModel
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
import logging
import tensorflow as tf
import torch
from nltk.tokenize import sent_tokenize
import faiss
import numpy as np

logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)

In [80]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

file_path = '/Users/t-arvio.anandi/Downloads/Train Label - Sheet1 (2).csv'
df = pd.read_csv(file_path)

x_data_list = df['x_data'].tolist()
y_data_list = df['y_data'].tolist()

# Tokenize and encode
encoding = tokenizer(x_data_list, padding='max_length', truncation=True, return_tensors='tf', max_length=40)
x_input_ids = encoding['input_ids']
x_attention_mask = encoding['attention_mask']

# Pass the token IDs through BERT to get embeddings
x_embeddings = bert_model(x_input_ids, attention_mask=x_attention_mask)[0].numpy()  # Shape: (batch_size, sequence_length, hidden_size)
# hidden_size is 768 for BERT-base, but we need 50 features

# Desired size for the last dimension
target_size = 50

# Pad or truncate the last dimension
if x_embeddings.shape[2] < target_size:
    # Pad the last dimension
    padding = target_size - x_embeddings.shape[2]
    x_embeddings_padded = np.pad(x_embeddings, ((0, 0), (0, 0), (0, padding)), 'constant')
else:
    # Truncate the last dimension
    x_embeddings_padded = x_embeddings[:, :, :target_size]


# Convert y_data_list to strings and handle NaNs
def clean_label(label):
    if pd.isna(label):
        return [0]  # Replace NaN with a default value
    try:
        return list(map(int, label.split()))
    except:
        return [0]  # Handle any other conversion issues

y_sequences = [clean_label(label) for label in y_data_list]

# Pad y_sequences
y_sequences_padded = pad_sequences(y_sequences, maxlen=40, padding='post', truncating='post', value=0)  # Using -1 or another placeholder for padding

# Verify shapes
print("Padded X Shape:", x_embeddings.shape)
print("Padded Y Shape:", y_sequences_padded.shape)

# Ensure the shapes are (91, 40, 50)
assert x_embeddings.shape[0] == 91  # Assuming 91 samples in total
assert x_embeddings.shape[1] == 40  # Sequence length
print(x_embeddings.shape[2])  # Embedding size
assert y_sequences_padded.shape[0] == 91
assert y_sequences_padded.shape[1] == 40
print(y_sequences_padded.shape[1])  # Adjust this based on the y labels’ actual format


/Users/t-arvio.anandi/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS

Padded X Shape: (91, 40, 768)
Padded Y Shape: (91, 40)
768
40


In [61]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input layer to match the shape of your data
inputs = layers.Input(shape=(40, 768))  # sequence_length=40, embedding_dim=768

# LSTM layer
lstm_out = layers.LSTM(64, return_sequences=True)(inputs)

# TimeDistributed Dense layer for classification
outputs = layers.TimeDistributed(layers.Dense(7, activation='softmax'))(lstm_out)

# Define the model
model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 40, 768)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 40, 64)         │       213,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 40, 7)          │           455 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 213,703 (834.78 KB)

 Trainable params: 213,703 (834.78 KB)

 Non-trainable params: 0 (0.00 B)

None


In [79]:
print(y_sequences_padded)

[[ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 ...
 [ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]]


In [81]:
from sklearn.model_selection import train_test_split
# Assuming you have more data, split into train/test
X_train, X_test, Y_train, Y_test = train_test_split(x_input_ids, y_sequences_padded, test_size=0.2)

# Fit the model
model.fit(x_input_ids, y_sequences_padded, epochs=3, batch_size=32)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 1, 15, 72, 43, 28, 79, 14, 49, 81, 89, 58, 35, 39, 12, 68, 47, 66,
       82, 42, 18, 45, 56,  0, 80, 10, 23,  9, 46,  6,  8, 71, 63, 67, 50,
        2, 76, 77, 33, 86, 26, 25, 24, 16, 85, 60, 84, 36, 19, 83, 75, 55,
       70, 17,  4,  3, 74, 29, 65, 32, 90, 88, 51, 40, 11, 52, 62, 41, 22,
       48, 61, 44, 37])

In [74]:
# Convert lists to NumPy arrays
x_input_ids_np = np.array(x_input_ids)
y_labels_np = np.array(y_labels)

# Print shapes to verify
print("Shape of x_input_ids:", x_input_ids_np.shape)
print("Shape of y_labels:", y_labels_np.shape)

Shape of x_input_ids: (91, 40)
Shape of y_labels: (25,)


In [83]:
# Example inference on new data
new_sentence = "What is your name?"
new_tokens = tokenizer.tokenize(new_sentence)
new_input_ids = tokenizer.convert_tokens_to_ids(new_tokens)
new_input_ids = pad_sequences([new_input_ids], maxlen=max_len, padding='post', truncating='post')

# Predict
predictions = model.predict(new_input_ids)
predicted_labels = tf.argmax(predictions, axis=-1)

print("Predicted Labels:", predicted_labels)

ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 40, 768), found shape=(1, 20)